In [79]:
import csv
import pandas as pd
import numpy as np
import math
import scipy
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [80]:
appVolDat = pd.read_csv('filingVolumes.csv')
prediction = appVolDat.iloc[1405]
appVolDat = appVolDat.iloc[1320:1404]
appVolDat = appVolDat.reset_index(drop=True)
maximum = max(appVolDat.volume)
appVolDat['vol'] = appVolDat.volume/maximum

trend1 = pd.read_csv('trend_CostOfPatent.csv').trends.iloc[72:156].reset_index(drop=True)/100
appVolDat['trend1'] = trend1#(trend1 - np.mean(trend1)) / np.std(trend1)

appVolDat

,month,volume,vol,trend1
0,1/1/2010,31365,0.377523,0.52
1,2/1/2010,33237,0.400055,0.36
2,3/1/2010,43055,0.518229,0.57
3,4/1/2010,38781,0.466785,0.30
4,5/1/2010,36019,0.433541,0.44
5,6/1/2010,42086,0.506566,0.59
6,7/1/2010,36667,0.441340,0.31
7,8/1/2010,37406,0.450235,0.33
8,9/1/2010,40013,0.481614,0.22
9,10/1/2010,38245,0.460334,0.21


In [81]:
for i in range(1,13):
    appVolDat['volShift'+str(i)] = appVolDat.vol.shift(i)
appVolDat = appVolDat.iloc[12:]

In [82]:
def func(X,a,b,c,d,e,f,g,h,i,j,k,l,m):
    return a*np.log(X['volShift1']) + b*np.log(X['volShift2']) + c*np.log(X['volShift3']) + \
d*np.log(X['volShift4']) + e*np.log(X['volShift5']) + f*np.log(X['volShift6']) + \
g*np.log(X['volShift7']) + h*np.log(X['volShift8']) + i*np.log(X['volShift9']) + \
j*np.log(X['volShift10']) + k*np.log(X['volShift11']) + l*np.log(X['volShift12']) + \
m*np.log(X['trend1'])

In [83]:
y = np.array(appVolDat['vol'])
fitParams, fitCovariances = curve_fit(func, appVolDat, y)
print(fitParams)

[ 0.14519829  0.08945291  0.17266047 -0.03436549  0.08106038  0.09867331
  0.02421089 -0.14721051 -0.06973218 -0.42569839 -0.33276707 -0.1657991
 -0.14196103]


In [84]:
ypred = fitParams[0]*appVolDat['volShift1'] + fitParams[1]*appVolDat['volShift2'] + fitParams[2]*appVolDat['volShift3'] + \
fitParams[3]*appVolDat['volShift4'] + fitParams[4]*appVolDat['volShift5'] + fitParams[5]*appVolDat['volShift6'] \
+ fitParams[6]*appVolDat['volShift7'] + fitParams[7]*appVolDat['volShift8'] + fitParams[8]*appVolDat['volShift9'] \
+ fitParams[9]*appVolDat['volShift10'] + fitParams[10]*appVolDat['volShift11'] + fitParams[11]*appVolDat['volShift12'] \
+ fitParams[12]*np.log(appVolDat['trend1'])

In [85]:
appVolDat.insert(3, 'volPred', np.array([10**x for x in ypred]))
print(ypred)

12   -0.147539
13   -0.211185
14   -0.112190
15   -0.118914
16   -0.136334
17   -0.145326
18   -0.140195
19   -0.176158
20   -0.093287
21   -0.198558
22   -0.174387
23   -0.065850
24   -0.195430
25   -0.222472
26   -0.186735
27   -0.189560
28   -0.154864
29   -0.158020
30   -0.125239
31   -0.156369
32   -0.189992
33   -0.129638
34   -0.171438
35   -0.073119
36   -0.167918
37   -0.199310
38   -0.210800
39   -0.188562
40   -0.172558
41   -0.102369
        ...   
54   -0.183994
55   -0.134620
56   -0.172118
57   -0.236769
58   -0.247800
59   -0.222634
60   -0.287965
61   -0.316804
62   -0.261640
63   -0.335261
64   -0.235816
65   -0.191156
66   -0.201311
67   -0.260998
68   -0.199978
69   -0.259578
70   -0.213398
71   -0.211001
72   -0.178672
73   -0.282441
74   -0.258745
75   -0.291502
76   -0.250606
77   -0.256077
78   -0.260544
79   -0.206093
80   -0.173156
81   -0.216420
82   -0.180485
83   -0.143045
Length: 72, dtype: float64


In [86]:
appVolDat['vol'] = appVolDat['vol'] * maximum
appVolDat['volPred'] = appVolDat['volPred'] * maximum
appVolDat

/Users/vishnumurthy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/vishnumurthy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,month,volume,vol,volPred,trend1,volShift1,volShift2,volShift3,volShift4,volShift5,volShift6,volShift7,volShift8,volShift9,volShift10,volShift11,volShift12
12,1/1/2011,35259,35259.0,59151.020042,0.53,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541,0.466785,0.518229,0.400055,0.377523
13,2/1/2011,35057,35057.0,51087.775707,0.64,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541,0.466785,0.518229,0.400055
14,3/1/2011,45603,45603.0,64167.041421,0.37,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541,0.466785,0.518229
15,4/1/2011,38163,38163.0,63181.228591,0.35,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541,0.466785
16,5/1/2011,38934,38934.0,60697.089607,0.43,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541
17,6/1/2011,43581,43581.0,59453.268259,0.53,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566
18,7/1/2011,35761,35761.0,60159.889730,0.43,0.524560,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340
19,8/1/2011,41170,41170.0,55378.842337,0.51,0.430435,0.524560,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235
20,9/1/2011,47684,47684.0,67021.530451,0.35,0.495540,0.430435,0.524560,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614
21,10/1/2011,37863,37863.0,52594.922098,0.60,0.573946,0.495540,0.430435,0.524560,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334


In [87]:
print("Training Error: ", np.sum(np.absolute(appVolDat['volPred']-appVolDat['vol'])) / len(appVolDat))

Training Error:  14932.829048581272
